In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
import re

In [ ]:
feature_ID_list = pd.read_csv('./Feature ID list for shock(MIMIC-4).csv')
feature_name = feature_ID_list['Feature name'].tolist()
feature_ID_list.set_index('Feature name', inplace=True)

inputevents = pd.read_csv('./inputevents.csv')
patients = pd.read_csv('./icustays.csv')

inputevents = pd.merge(inputevents, patients, on=['subject_id','hadm_id','stay_id'], how='left')
inputevents.reset_index(drop=True, inplace=True)

condition1= (inputevents['intime'] <= inputevents['starttime'])
condition2 = (inputevents['outtime'] >= inputevents['starttime'])

inputevents = inputevents[condition1 & condition2]
inputevents = inputevents.sort_values(by=['subject_id','hadm_id','stay_id','starttime'], ascending=[True,True,True,True])
inputevents.reset_index(drop=True, inplace=True)

Ketamine

In [ ]:
Ketamine_id_ls = feature_ID_list['inputevents'][feature_name[14]]
Ketamine_id_ls = [int(x) for x in Ketamine_id_ls.split(',')]

Ketamine = inputevents[inputevents['itemid'].isin(Ketamine_id_ls)]
Ketamine['amount'] = 1
Ketamine.reset_index(drop=True, inplace=True)

Ketamine = Ketamine[Ketamine['amount'].notna()]
condition1 = Ketamine['amount']>=0 
Ketamine = Ketamine[(condition1)]
Ketamine.reset_index(drop=True, inplace=True)
Ketamine = Ketamine.sort_values(by=['subject_id','hadm_id','stay_id','starttime'], ascending=[True,True,True,True])
Ketamine = Ketamine[['subject_id','hadm_id','stay_id', 'intime', 'outtime', 'starttime', 'endtime', 'amount']]
Ketamine.reset_index(drop=True, inplace=True)

Ketamine['starttime'] = pd.to_datetime(Ketamine['starttime'])
Ketamine['endtime'] = pd.to_datetime(Ketamine['endtime'])
Ketamine['outtime'] = pd.to_datetime(Ketamine['outtime'])
Ketamine['intime'] = pd.to_datetime(Ketamine['intime'])

Ketamine = Ketamine[Ketamine['starttime']<=Ketamine['endtime']]
print(Ketamine.shape)
Ketamine.reset_index(drop=True, inplace=True)

In [ ]:
def resampling_func2(data):
    feature_name ='Ketamine'

    column = ['subject_id','hadm_id','stay_id','time', feature_name]

    x = data.reset_index(drop=True)
    
    #x.set_index(keys='starttime', inplace=True, drop=False)

    date_range = pd.date_range(x['intime'][0], x['outtime'][0], freq='10T', inclusive='right')

    new_df = pd.DataFrame(index=date_range, columns=column)

    new_df['subject_id'] = x['subject_id'][0]
    new_df['hadm_id'] = x['hadm_id'][0]
    new_df['stay_id'] = x['stay_id'][0]
    new_df['time'] = new_df.index
    new_df[feature_name] = 0
    
    new_df = new_df.reset_index(drop=True)
    
    for i in range(len(new_df)):
        if i == 0:
            condition1 = (date_range[i]>=x['starttime'])
            condition2 = (date_range[i]>=x['endtime'])
            sa1 = x[condition1 & condition2]

            condition3 = (date_range[i]>=x['starttime'])
            condition4 = (date_range[i]<x['endtime'])
            sa2 = x[condition3 & condition4]

            if (sa2.empty)==False :
                sa2.reset_index(drop=True, inplace=True)
                sa2['duration'] = 1

                for p in range(len(sa2)):
                    duration1 = sa2['endtime'][p] - sa2['starttime'][p]
                    duration2 = date_range[i] - sa2['starttime'][p]
                    if (duration1 != 0) & (duration2 != 0) & (duration1 > duration2):
                        sa2['duration'][p] = duration2/duration1
                    else :
                        sa2['duration'][p] = 1

                sa2['amount'] = sa2['amount']*sa2['duration']
            
                if (sa1.empty)==False :
                    sa1_sum = sa1['amount'].sum()
                    sa2_sum = sa2['amount'].sum()

                    new_df[feature_name][i] = sa1_sum + sa2_sum

                else :
                    sa2_sum = sa2['amount'].sum()

                    new_df[feature_name][i] = sa2_sum

            else :
                if (sa1.empty)==False :
                    sa1_sum = sa1['amount'].sum()

                    new_df[feature_name][i] = sa1_sum

                else :
                    new_df[feature_name][i] = 0

        else :
            condition1 = ((date_range[i]>=x['starttime']) & (date_range[i-1]<x['starttime']))
            condition2 = ((date_range[i]>=x['endtime']) & (date_range[i-1]<x['endtime']))
            sa1 = x[condition1 & condition2]

            condition3 = ((date_range[i]>=x['starttime']) & (date_range[i-1]<x['starttime']))
            condition4 = ((date_range[i]<x['endtime']))
            sa2 = x[condition3 & condition4]
            
            condition5 = ((date_range[i-1]>=x['starttime']))
            condition6 = ((date_range[i]>=x['endtime']) & (date_range[i-1]<x['endtime']))
            sa3 = x[condition5 & condition6]

            condition7 = ((date_range[i-1]>=x['starttime']))
            condition8 = ((date_range[i]<x['endtime']))
            sa4 = x[condition7 & condition8]
            
            if (sa4.empty)==False :
                sa4.reset_index(drop=True, inplace=True)
                sa4['duration'] = 1

                for p in range(len(sa4)):
                    duration1 = sa4['endtime'][p] - sa4['starttime'][p]
                    duration2 = sa4['endtime'][p] - date_range[i]
                    duration3 = date_range[i-1] - sa4['starttime'][p]

                    if (duration1 != 0) & (duration2 != 0) & (duration3 != 0) & (duration1 > duration2) & (duration1 > duration3):
                        sa4['duration'][p] = 1 - duration2/duration1 - duration3/duration1
                        
                    else :
                        sa4['duration'][p] = 1

                sa4['amount'] = sa4['amount']*sa4['duration']

                if (sa2.empty)==False :
                    sa2.reset_index(drop=True, inplace=True)
                    sa2['duration'] = 1

                    for p in range(len(sa2)):
                        duration1 = sa2['endtime'][p] - sa2['starttime'][p]
                        duration2 = date_range[i] - sa2['starttime'][p]

                        if (duration1 != 0) & (duration2 != 0) & (duration1 > duration2):
                            sa2['duration'][p] = duration2/duration1
                        else :
                            sa2['duration'][p] = 1

                    sa2['amount'] = sa2['amount']*sa2['duration']


                    if (sa3.empty)==False :
                        sa3.reset_index(drop=True, inplace=True)
                        sa3['duration'] = 1

                        for p in range(len(sa3)):
                            duration3 = sa3['endtime'][p] - sa3['starttime'][p]
                            duration4 = sa3['endtime'][p] - date_range[i]
                            duration5 = date_range[i] - sa3['starttime'][p]

                            if (duration3 != 0) & (duration4 != 0) :
                                sa3['duration'][p] = 1 - duration4/duration3 - duration5/duration3
                            else :
                                sa3['duration'][p] = 1

                        sa3['amount'] = sa3['amount']*sa3['duration']

                
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa2_sum = sa2['amount'].sum()
                            sa3_sum = sa3['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa2_sum + sa3_sum + sa4_sum
                        else :
                            sa2_sum = sa2['amount'].sum()
                            sa3_sum = sa3['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa2_sum + sa3_sum + sa4_sum

                    else :
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa2_sum = sa2['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa2_sum + sa4_sum
                        else :
                            sa2_sum = sa2['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa2_sum + sa4_sum



                else :
                    if (sa3.empty)==False :
                        sa3.reset_index(drop=True, inplace=True)
                        sa3['duration'] = 1

                        for p in range(len(sa3)):
                            duration3 = sa3['endtime'][p] - sa3['starttime'][p]
                            duration4 = date_range[i] - sa3['starttime'][p]

                            if (duration3 != 0) & (duration4 != 0) :
                                sa3['duration'][p] = duration4/duration3
                            else :
                                sa3['duration'][p] = 1

                        sa3['amount'] = sa3['amount']*sa3['duration']

                
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa3_sum = sa3['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa3_sum + sa4_sum
                        else :
                            sa3_sum = sa3['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa3_sum + sa4_sum

                    else :
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa4_sum
                        else :
                            sa4_sum = sa4['amount'].sum()
                            
                            new_df[feature_name][i] = sa4_sum

            else:
                if (sa2.empty)==False :
                    sa2.reset_index(drop=True, inplace=True)
                    sa2['duration'] = 1

                    for p in range(len(sa2)):
                        duration1 = sa2['endtime'][p] - sa2['starttime'][p]
                        duration2 = date_range[i] - sa2['starttime'][p]

                        if (duration1 != 0) & (duration2 != 0) & (duration1 > duration2):
                            sa2['duration'][p] = duration2/duration1
                        else :
                            sa2['duration'][p] = 1

                    sa2['amount'] = sa2['amount']*sa2['duration']


                    if (sa3.empty)==False :
                        sa3.reset_index(drop=True, inplace=True)
                        sa3['duration'] = 1

                        for p in range(len(sa3)):
                            duration3 = sa3['endtime'][p] - sa3['starttime'][p]
                            duration4 = sa3['endtime'][p] - date_range[i]
                            duration5 = date_range[i] - sa3['starttime'][p]

                            if (duration3 != 0) & (duration4 != 0) :
                                sa3['duration'][p] = 1 - duration4/duration3 - duration5/duration3
                            else :
                                sa3['duration'][p] = 1

                        sa3['amount'] = sa3['amount']*sa3['duration']

                
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa2_sum = sa2['amount'].sum()
                            sa3_sum = sa3['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa2_sum + sa3_sum
                        else :
                            sa2_sum = sa2['amount'].sum()
                            sa3_sum = sa3['amount'].sum()

                            new_df[feature_name][i] = sa2_sum + sa3_sum

                    else :
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa2_sum = sa2['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa2_sum
                        else :
                            sa2_sum = sa2['amount'].sum()

                            new_df[feature_name][i] = sa2_sum 



                else :
                    if (sa3.empty)==False :
                        sa3.reset_index(drop=True, inplace=True)
                        sa3['duration'] = 1

                        for p in range(len(sa3)):
                            duration3 = sa3['endtime'][p] - sa3['starttime'][p]
                            duration4 = date_range[i] - sa3['starttime'][p]

                            if (duration3 != 0) & (duration4 != 0) :
                                sa3['duration'][p] = duration4/duration3
                            else :
                                sa3['duration'][p] = 1

                        sa3['amount'] = sa3['amount']*sa3['duration']
                
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa3_sum = sa3['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa3_sum
                        else :
                            sa3_sum = sa3['amount'].sum()

                            new_df[feature_name][i] = sa3_sum

                    else :
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()

                            new_df[feature_name][i] = sa1_sum
                        else :  
                            new_df[feature_name][i] = 0

    new_df = new_df[['subject_id', 'hadm_id', 'stay_id', 'time', feature_name]]
    new_df = new_df.reset_index(drop=True)
  
    return new_df

result_Ketamine = Ketamine.groupby('stay_id').apply(resampling_func2)
result_Ketamine.reset_index(drop=True, inplace=True)

result_Ketamine_index = result_Ketamine[result_Ketamine['Ketamine']>0].index
result_Ketamine['Ketamine'][result_Ketamine_index] = 1
result_Ketamine = result_Ketamine.reset_index(drop=True)

print('---------------------------------------------------------------------------------')
result_Ketamine.to_csv('./10min Ketamine.csv', index=False)

Propofol

In [ ]:
Propofol_id_ls = feature_ID_list['inputevents'][feature_name[15]]
Propofol_id_ls = [int(x) for x in Propofol_id_ls.split(',')]

Propofol = inputevents[inputevents['itemid'].isin(Propofol_id_ls)]
Propofol.reset_index(drop=True, inplace=True)

Propofol = Propofol[Propofol['amount'].notna()]
condition1 = Propofol['amount']>=0 
Propofol = Propofol[(condition1)]
Propofol.reset_index(drop=True, inplace=True)
Propofol = Propofol.sort_values(by=['subject_id', 'hadm_id', 'stay_id', 'starttime'], ascending=[True,True,True,True])
Propofol = Propofol[['subject_id', 'hadm_id', 'stay_id', 'intime', 'outtime', 'starttime', 'endtime', 'amount']]
Propofol.reset_index(drop=True, inplace=True)

Propofol['starttime'] = pd.to_datetime(Propofol['starttime'])
Propofol['endtime'] = pd.to_datetime(Propofol['endtime'])
Propofol['outtime'] = pd.to_datetime(Propofol['outtime'])
Propofol['intime'] = pd.to_datetime(Propofol['intime'])

Propofol = Propofol[Propofol['starttime']<=Propofol['endtime']]
print(Propofol.shape)
Propofol.reset_index(drop=True, inplace=True)

In [ ]:
def resampling_func2(data):
    feature_name ='Propofol'

    column = ['subject_id','hadm_id','stay_id','time', feature_name]

    x = data.reset_index(drop=True)
    
    #x.set_index(keys='starttime', inplace=True, drop=False)

    date_range = pd.date_range(x['intime'][0], x['outtime'][0], freq='10T', inclusive='right')

    new_df = pd.DataFrame(index=date_range, columns=column)

    new_df['subject_id'] = x['subject_id'][0]
    new_df['hadm_id'] = x['hadm_id'][0]
    new_df['stay_id'] = x['stay_id'][0]
    new_df['time'] = new_df.index
    new_df[feature_name] = 0
    
    new_df = new_df.reset_index(drop=True)
    
    for i in range(len(new_df)):
        if i == 0:
            condition1 = (date_range[i]>=x['starttime'])
            condition2 = (date_range[i]>=x['endtime'])
            sa1 = x[condition1 & condition2]

            condition3 = (date_range[i]>=x['starttime'])
            condition4 = (date_range[i]<x['endtime'])
            sa2 = x[condition3 & condition4]

            if (sa2.empty)==False :
                sa2.reset_index(drop=True, inplace=True)
                sa2['duration'] = 1

                for p in range(len(sa2)):
                    duration1 = sa2['endtime'][p] - sa2['starttime'][p]
                    duration2 = date_range[i] - sa2['starttime'][p]
                    if (duration1 != 0) & (duration2 != 0) & (duration1 > duration2):
                        sa2['duration'][p] = duration2/duration1
                    else :
                        sa2['duration'][p] = 1

                sa2['amount'] = sa2['amount']*sa2['duration']
            
                if (sa1.empty)==False :
                    sa1_sum = sa1['amount'].sum()
                    sa2_sum = sa2['amount'].sum()

                    new_df[feature_name][i] = sa1_sum + sa2_sum

                else :
                    sa2_sum = sa2['amount'].sum()

                    new_df[feature_name][i] = sa2_sum

            else :
                if (sa1.empty)==False :
                    sa1_sum = sa1['amount'].sum()

                    new_df[feature_name][i] = sa1_sum

                else :
                    new_df[feature_name][i] = 0

        else :
            condition1 = ((date_range[i]>=x['starttime']) & (date_range[i-1]<x['starttime']))
            condition2 = ((date_range[i]>=x['endtime']) & (date_range[i-1]<x['endtime']))
            sa1 = x[condition1 & condition2]

            condition3 = ((date_range[i]>=x['starttime']) & (date_range[i-1]<x['starttime']))
            condition4 = ((date_range[i]<x['endtime']))
            sa2 = x[condition3 & condition4]
            
            condition5 = ((date_range[i-1]>=x['starttime']))
            condition6 = ((date_range[i]>=x['endtime']) & (date_range[i-1]<x['endtime']))
            sa3 = x[condition5 & condition6]

            condition7 = ((date_range[i-1]>=x['starttime']))
            condition8 = ((date_range[i]<x['endtime']))
            sa4 = x[condition7 & condition8]
            
            if (sa4.empty)==False :
                sa4.reset_index(drop=True, inplace=True)
                sa4['duration'] = 1

                for p in range(len(sa4)):
                    duration1 = sa4['endtime'][p] - sa4['starttime'][p]
                    duration2 = sa4['endtime'][p] - date_range[i]
                    duration3 = date_range[i-1] - sa4['starttime'][p]

                    if (duration1 != 0) & (duration2 != 0) & (duration3 != 0) & (duration1 > duration2) & (duration1 > duration3):
                        sa4['duration'][p] = 1 - duration2/duration1 - duration3/duration1
                        
                    else :
                        sa4['duration'][p] = 1

                sa4['amount'] = sa4['amount']*sa4['duration']

                if (sa2.empty)==False :
                    sa2.reset_index(drop=True, inplace=True)
                    sa2['duration'] = 1

                    for p in range(len(sa2)):
                        duration1 = sa2['endtime'][p] - sa2['starttime'][p]
                        duration2 = date_range[i] - sa2['starttime'][p]

                        if (duration1 != 0) & (duration2 != 0) & (duration1 > duration2):
                            sa2['duration'][p] = duration2/duration1
                        else :
                            sa2['duration'][p] = 1

                    sa2['amount'] = sa2['amount']*sa2['duration']


                    if (sa3.empty)==False :
                        sa3.reset_index(drop=True, inplace=True)
                        sa3['duration'] = 1

                        for p in range(len(sa3)):
                            duration3 = sa3['endtime'][p] - sa3['starttime'][p]
                            duration4 = sa3['endtime'][p] - date_range[i]
                            duration5 = date_range[i] - sa3['starttime'][p]

                            if (duration3 != 0) & (duration4 != 0) :
                                sa3['duration'][p] = 1 - duration4/duration3 - duration5/duration3
                            else :
                                sa3['duration'][p] = 1

                        sa3['amount'] = sa3['amount']*sa3['duration']

                
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa2_sum = sa2['amount'].sum()
                            sa3_sum = sa3['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa2_sum + sa3_sum + sa4_sum
                        else :
                            sa2_sum = sa2['amount'].sum()
                            sa3_sum = sa3['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa2_sum + sa3_sum + sa4_sum

                    else :
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa2_sum = sa2['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa2_sum + sa4_sum
                        else :
                            sa2_sum = sa2['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa2_sum + sa4_sum



                else :
                    if (sa3.empty)==False :
                        sa3.reset_index(drop=True, inplace=True)
                        sa3['duration'] = 1

                        for p in range(len(sa3)):
                            duration3 = sa3['endtime'][p] - sa3['starttime'][p]
                            duration4 = date_range[i] - sa3['starttime'][p]

                            if (duration3 != 0) & (duration4 != 0) :
                                sa3['duration'][p] = duration4/duration3
                            else :
                                sa3['duration'][p] = 1

                        sa3['amount'] = sa3['amount']*sa3['duration']

                
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa3_sum = sa3['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa3_sum + sa4_sum
                        else :
                            sa3_sum = sa3['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa3_sum + sa4_sum

                    else :
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa4_sum
                        else :
                            sa4_sum = sa4['amount'].sum()
                            
                            new_df[feature_name][i] = sa4_sum

            else:
                if (sa2.empty)==False :
                    sa2.reset_index(drop=True, inplace=True)
                    sa2['duration'] = 1

                    for p in range(len(sa2)):
                        duration1 = sa2['endtime'][p] - sa2['starttime'][p]
                        duration2 = date_range[i] - sa2['starttime'][p]

                        if (duration1 != 0) & (duration2 != 0) & (duration1 > duration2):
                            sa2['duration'][p] = duration2/duration1
                        else :
                            sa2['duration'][p] = 1

                    sa2['amount'] = sa2['amount']*sa2['duration']


                    if (sa3.empty)==False :
                        sa3.reset_index(drop=True, inplace=True)
                        sa3['duration'] = 1

                        for p in range(len(sa3)):
                            duration3 = sa3['endtime'][p] - sa3['starttime'][p]
                            duration4 = sa3['endtime'][p] - date_range[i]
                            duration5 = date_range[i] - sa3['starttime'][p]

                            if (duration3 != 0) & (duration4 != 0) :
                                sa3['duration'][p] = 1 - duration4/duration3 - duration5/duration3
                            else :
                                sa3['duration'][p] = 1

                        sa3['amount'] = sa3['amount']*sa3['duration']

                
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa2_sum = sa2['amount'].sum()
                            sa3_sum = sa3['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa2_sum + sa3_sum
                        else :
                            sa2_sum = sa2['amount'].sum()
                            sa3_sum = sa3['amount'].sum()

                            new_df[feature_name][i] = sa2_sum + sa3_sum

                    else :
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa2_sum = sa2['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa2_sum
                        else :
                            sa2_sum = sa2['amount'].sum()

                            new_df[feature_name][i] = sa2_sum 



                else :
                    if (sa3.empty)==False :
                        sa3.reset_index(drop=True, inplace=True)
                        sa3['duration'] = 1

                        for p in range(len(sa3)):
                            duration3 = sa3['endtime'][p] - sa3['starttime'][p]
                            duration4 = date_range[i] - sa3['starttime'][p]

                            if (duration3 != 0) & (duration4 != 0) :
                                sa3['duration'][p] = duration4/duration3
                            else :
                                sa3['duration'][p] = 1

                        sa3['amount'] = sa3['amount']*sa3['duration']
                
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa3_sum = sa3['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa3_sum
                        else :
                            sa3_sum = sa3['amount'].sum()

                            new_df[feature_name][i] = sa3_sum

                    else :
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()

                            new_df[feature_name][i] = sa1_sum
                        else :  
                            new_df[feature_name][i] = 0

    new_df = new_df[['subject_id', 'hadm_id', 'stay_id', 'time', feature_name]]
    new_df = new_df.reset_index(drop=True)
  
    return new_df

result_Propofol = Propofol.groupby('stay_id').apply(resampling_func2)
result_Propofol.reset_index(drop=True, inplace=True)

result_Propofol_index = result_Propofol[result_Propofol['Propofol']>0].index
result_Propofol['Propofol'][result_Propofol_index] = 1
result_Propofol = result_Propofol.reset_index(drop=True)

print('---------------------------------------------------------------------------------')
result_Propofol.to_csv('./10min Propofol.csv', index=False)

Midazolam

In [ ]:
Midazolam_id_ls = feature_ID_list['inputevents'][feature_name[16]]
Midazolam_id_ls = [int(x) for x in Midazolam_id_ls.split(',')]

Midazolam = inputevents[inputevents['itemid'].isin(Midazolam_id_ls)]
Midazolam.reset_index(drop=True, inplace=True)

Midazolam = Midazolam[Midazolam['amount'].notna()]
condition1 = Midazolam['amount']>=0 
Midazolam = Midazolam[(condition1)]
Midazolam.reset_index(drop=True, inplace=True)
Midazolam = Midazolam.sort_values(by=['subject_id', 'hadm_id', 'stay_id', 'starttime'], ascending=[True,True,True,True])
Midazolam = Midazolam[['subject_id', 'hadm_id', 'stay_id', 'intime', 'outtime', 'starttime', 'endtime', 'amount']]
Midazolam.reset_index(drop=True, inplace=True)

Midazolam['starttime'] = pd.to_datetime(Midazolam['starttime'])
Midazolam['endtime'] = pd.to_datetime(Midazolam['endtime'])
Midazolam['outtime'] = pd.to_datetime(Midazolam['outtime'])
Midazolam['intime'] = pd.to_datetime(Midazolam['intime'])

Midazolam = Midazolam[Midazolam['starttime']<=Midazolam['endtime']]
print(Midazolam.shape)
Midazolam.reset_index(drop=True, inplace=True)

In [ ]:
def resampling_func2(data):
    feature_name ='Midazolam'

    column = ['subject_id','hadm_id','stay_id','time', feature_name]

    x = data.reset_index(drop=True)
    
    #x.set_index(keys='starttime', inplace=True, drop=False)

    date_range = pd.date_range(x['intime'][0], x['outtime'][0], freq='10T', inclusive='right')

    new_df = pd.DataFrame(index=date_range, columns=column)

    new_df['subject_id'] = x['subject_id'][0]
    new_df['hadm_id'] = x['hadm_id'][0]
    new_df['stay_id'] = x['stay_id'][0]
    new_df['time'] = new_df.index
    new_df[feature_name] = 0
    
    new_df = new_df.reset_index(drop=True)
    
    for i in range(len(new_df)):
        if i == 0:
            condition1 = (date_range[i]>=x['starttime'])
            condition2 = (date_range[i]>=x['endtime'])
            sa1 = x[condition1 & condition2]

            condition3 = (date_range[i]>=x['starttime'])
            condition4 = (date_range[i]<x['endtime'])
            sa2 = x[condition3 & condition4]

            if (sa2.empty)==False :
                sa2.reset_index(drop=True, inplace=True)
                sa2['duration'] = 1

                for p in range(len(sa2)):
                    duration1 = sa2['endtime'][p] - sa2['starttime'][p]
                    duration2 = date_range[i] - sa2['starttime'][p]
                    if (duration1 != 0) & (duration2 != 0) & (duration1 > duration2):
                        sa2['duration'][p] = duration2/duration1
                    else :
                        sa2['duration'][p] = 1

                sa2['amount'] = sa2['amount']*sa2['duration']
            
                if (sa1.empty)==False :
                    sa1_sum = sa1['amount'].sum()
                    sa2_sum = sa2['amount'].sum()

                    new_df[feature_name][i] = sa1_sum + sa2_sum

                else :
                    sa2_sum = sa2['amount'].sum()

                    new_df[feature_name][i] = sa2_sum

            else :
                if (sa1.empty)==False :
                    sa1_sum = sa1['amount'].sum()

                    new_df[feature_name][i] = sa1_sum

                else :
                    new_df[feature_name][i] = 0

        else :
            condition1 = ((date_range[i]>=x['starttime']) & (date_range[i-1]<x['starttime']))
            condition2 = ((date_range[i]>=x['endtime']) & (date_range[i-1]<x['endtime']))
            sa1 = x[condition1 & condition2]

            condition3 = ((date_range[i]>=x['starttime']) & (date_range[i-1]<x['starttime']))
            condition4 = ((date_range[i]<x['endtime']))
            sa2 = x[condition3 & condition4]
            
            condition5 = ((date_range[i-1]>=x['starttime']))
            condition6 = ((date_range[i]>=x['endtime']) & (date_range[i-1]<x['endtime']))
            sa3 = x[condition5 & condition6]

            condition7 = ((date_range[i-1]>=x['starttime']))
            condition8 = ((date_range[i]<x['endtime']))
            sa4 = x[condition7 & condition8]
            
            if (sa4.empty)==False :
                sa4.reset_index(drop=True, inplace=True)
                sa4['duration'] = 1

                for p in range(len(sa4)):
                    duration1 = sa4['endtime'][p] - sa4['starttime'][p]
                    duration2 = sa4['endtime'][p] - date_range[i]
                    duration3 = date_range[i-1] - sa4['starttime'][p]

                    if (duration1 != 0) & (duration2 != 0) & (duration3 != 0) & (duration1 > duration2) & (duration1 > duration3):
                        sa4['duration'][p] = 1 - duration2/duration1 - duration3/duration1
                        
                    else :
                        sa4['duration'][p] = 1

                sa4['amount'] = sa4['amount']*sa4['duration']

                if (sa2.empty)==False :
                    sa2.reset_index(drop=True, inplace=True)
                    sa2['duration'] = 1

                    for p in range(len(sa2)):
                        duration1 = sa2['endtime'][p] - sa2['starttime'][p]
                        duration2 = date_range[i] - sa2['starttime'][p]

                        if (duration1 != 0) & (duration2 != 0) & (duration1 > duration2):
                            sa2['duration'][p] = duration2/duration1
                        else :
                            sa2['duration'][p] = 1

                    sa2['amount'] = sa2['amount']*sa2['duration']


                    if (sa3.empty)==False :
                        sa3.reset_index(drop=True, inplace=True)
                        sa3['duration'] = 1

                        for p in range(len(sa3)):
                            duration3 = sa3['endtime'][p] - sa3['starttime'][p]
                            duration4 = sa3['endtime'][p] - date_range[i]
                            duration5 = date_range[i] - sa3['starttime'][p]

                            if (duration3 != 0) & (duration4 != 0) :
                                sa3['duration'][p] = 1 - duration4/duration3 - duration5/duration3
                            else :
                                sa3['duration'][p] = 1

                        sa3['amount'] = sa3['amount']*sa3['duration']

                
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa2_sum = sa2['amount'].sum()
                            sa3_sum = sa3['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa2_sum + sa3_sum + sa4_sum
                        else :
                            sa2_sum = sa2['amount'].sum()
                            sa3_sum = sa3['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa2_sum + sa3_sum + sa4_sum

                    else :
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa2_sum = sa2['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa2_sum + sa4_sum
                        else :
                            sa2_sum = sa2['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa2_sum + sa4_sum



                else :
                    if (sa3.empty)==False :
                        sa3.reset_index(drop=True, inplace=True)
                        sa3['duration'] = 1

                        for p in range(len(sa3)):
                            duration3 = sa3['endtime'][p] - sa3['starttime'][p]
                            duration4 = date_range[i] - sa3['starttime'][p]

                            if (duration3 != 0) & (duration4 != 0) :
                                sa3['duration'][p] = duration4/duration3
                            else :
                                sa3['duration'][p] = 1

                        sa3['amount'] = sa3['amount']*sa3['duration']

                
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa3_sum = sa3['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa3_sum + sa4_sum
                        else :
                            sa3_sum = sa3['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa3_sum + sa4_sum

                    else :
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa4_sum
                        else :
                            sa4_sum = sa4['amount'].sum()
                            
                            new_df[feature_name][i] = sa4_sum

            else:
                if (sa2.empty)==False :
                    sa2.reset_index(drop=True, inplace=True)
                    sa2['duration'] = 1

                    for p in range(len(sa2)):
                        duration1 = sa2['endtime'][p] - sa2['starttime'][p]
                        duration2 = date_range[i] - sa2['starttime'][p]

                        if (duration1 != 0) & (duration2 != 0) & (duration1 > duration2):
                            sa2['duration'][p] = duration2/duration1
                        else :
                            sa2['duration'][p] = 1

                    sa2['amount'] = sa2['amount']*sa2['duration']


                    if (sa3.empty)==False :
                        sa3.reset_index(drop=True, inplace=True)
                        sa3['duration'] = 1

                        for p in range(len(sa3)):
                            duration3 = sa3['endtime'][p] - sa3['starttime'][p]
                            duration4 = sa3['endtime'][p] - date_range[i]
                            duration5 = date_range[i] - sa3['starttime'][p]

                            if (duration3 != 0) & (duration4 != 0) :
                                sa3['duration'][p] = 1 - duration4/duration3 - duration5/duration3
                            else :
                                sa3['duration'][p] = 1

                        sa3['amount'] = sa3['amount']*sa3['duration']

                
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa2_sum = sa2['amount'].sum()
                            sa3_sum = sa3['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa2_sum + sa3_sum
                        else :
                            sa2_sum = sa2['amount'].sum()
                            sa3_sum = sa3['amount'].sum()

                            new_df[feature_name][i] = sa2_sum + sa3_sum

                    else :
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa2_sum = sa2['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa2_sum
                        else :
                            sa2_sum = sa2['amount'].sum()

                            new_df[feature_name][i] = sa2_sum 



                else :
                    if (sa3.empty)==False :
                        sa3.reset_index(drop=True, inplace=True)
                        sa3['duration'] = 1

                        for p in range(len(sa3)):
                            duration3 = sa3['endtime'][p] - sa3['starttime'][p]
                            duration4 = date_range[i] - sa3['starttime'][p]

                            if (duration3 != 0) & (duration4 != 0) :
                                sa3['duration'][p] = duration4/duration3
                            else :
                                sa3['duration'][p] = 1

                        sa3['amount'] = sa3['amount']*sa3['duration']
                
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa3_sum = sa3['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa3_sum
                        else :
                            sa3_sum = sa3['amount'].sum()

                            new_df[feature_name][i] = sa3_sum

                    else :
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()

                            new_df[feature_name][i] = sa1_sum
                        else :  
                            new_df[feature_name][i] = 0

    new_df = new_df[['subject_id', 'hadm_id', 'stay_id', 'time', feature_name]]
    new_df = new_df.reset_index(drop=True)
  
    return new_df

result_Midazolam = Midazolam.groupby('stay_id').apply(resampling_func2)
result_Midazolam.reset_index(drop=True, inplace=True)

result_Midazolam_index = result_Midazolam[result_Midazolam['Midazolam']>0].index
result_Midazolam['Midazolam'][result_Midazolam_index] = 1
result_Midazolam = result_Midazolam.reset_index(drop=True)

print('---------------------------------------------------------------------------------')
result_Midazolam.to_csv('./10min Midazolam.csv', index=False)

Fentanyl

In [ ]:
Fentanyl_id_ls = feature_ID_list['inputevents'][feature_name[17]]
Fentanyl_id_ls = [int(x) for x in Fentanyl_id_ls.split(',')]

Fentanyl = inputevents[inputevents['itemid'].isin(Fentanyl_id_ls)]
Fentanyl.reset_index(drop=True, inplace=True)

Fentanyl = Fentanyl[Fentanyl['amount'].notna()]
condition1 = Fentanyl['amount']>=0 
Fentanyl = Fentanyl[(condition1)]
Fentanyl.reset_index(drop=True, inplace=True)
Fentanyl = Fentanyl.sort_values(by=['subject_id', 'hadm_id', 'stay_id', 'starttime'], ascending=[True,True,True,True])
Fentanyl = Fentanyl[['subject_id', 'hadm_id', 'stay_id', 'intime', 'outtime', 'starttime', 'endtime', 'amount']]
Fentanyl.reset_index(drop=True, inplace=True)

Fentanyl['starttime'] = pd.to_datetime(Fentanyl['starttime'])
Fentanyl['endtime'] = pd.to_datetime(Fentanyl['endtime'])
Fentanyl['outtime'] = pd.to_datetime(Fentanyl['outtime'])
Fentanyl['intime'] = pd.to_datetime(Fentanyl['intime'])

Fentanyl = Fentanyl[Fentanyl['starttime']<=Fentanyl['endtime']]
print(Fentanyl.shape)
Fentanyl.reset_index(drop=True, inplace=True)

In [ ]:
def resampling_func2(data):
    feature_name ='Fentanyl'

    column = ['subject_id','hadm_id','stay_id','time', feature_name]

    x = data.reset_index(drop=True)
    
    #x.set_index(keys='starttime', inplace=True, drop=False)

    date_range = pd.date_range(x['intime'][0], x['outtime'][0], freq='10T', inclusive='right')

    new_df = pd.DataFrame(index=date_range, columns=column)

    new_df['subject_id'] = x['subject_id'][0]
    new_df['hadm_id'] = x['hadm_id'][0]
    new_df['stay_id'] = x['stay_id'][0]
    new_df['time'] = new_df.index
    new_df[feature_name] = 0
    
    new_df = new_df.reset_index(drop=True)
    
    for i in range(len(new_df)):
        if i == 0:
            condition1 = (date_range[i]>=x['starttime'])
            condition2 = (date_range[i]>=x['endtime'])
            sa1 = x[condition1 & condition2]

            condition3 = (date_range[i]>=x['starttime'])
            condition4 = (date_range[i]<x['endtime'])
            sa2 = x[condition3 & condition4]

            if (sa2.empty)==False :
                sa2.reset_index(drop=True, inplace=True)
                sa2['duration'] = 1

                for p in range(len(sa2)):
                    duration1 = sa2['endtime'][p] - sa2['starttime'][p]
                    duration2 = date_range[i] - sa2['starttime'][p]
                    if (duration1 != 0) & (duration2 != 0) & (duration1 > duration2):
                        sa2['duration'][p] = duration2/duration1
                    else :
                        sa2['duration'][p] = 1

                sa2['amount'] = sa2['amount']*sa2['duration']
            
                if (sa1.empty)==False :
                    sa1_sum = sa1['amount'].sum()
                    sa2_sum = sa2['amount'].sum()

                    new_df[feature_name][i] = sa1_sum + sa2_sum

                else :
                    sa2_sum = sa2['amount'].sum()

                    new_df[feature_name][i] = sa2_sum

            else :
                if (sa1.empty)==False :
                    sa1_sum = sa1['amount'].sum()

                    new_df[feature_name][i] = sa1_sum

                else :
                    new_df[feature_name][i] = 0

        else :
            condition1 = ((date_range[i]>=x['starttime']) & (date_range[i-1]<x['starttime']))
            condition2 = ((date_range[i]>=x['endtime']) & (date_range[i-1]<x['endtime']))
            sa1 = x[condition1 & condition2]

            condition3 = ((date_range[i]>=x['starttime']) & (date_range[i-1]<x['starttime']))
            condition4 = ((date_range[i]<x['endtime']))
            sa2 = x[condition3 & condition4]
            
            condition5 = ((date_range[i-1]>=x['starttime']))
            condition6 = ((date_range[i]>=x['endtime']) & (date_range[i-1]<x['endtime']))
            sa3 = x[condition5 & condition6]

            condition7 = ((date_range[i-1]>=x['starttime']))
            condition8 = ((date_range[i]<x['endtime']))
            sa4 = x[condition7 & condition8]
            
            if (sa4.empty)==False :
                sa4.reset_index(drop=True, inplace=True)
                sa4['duration'] = 1

                for p in range(len(sa4)):
                    duration1 = sa4['endtime'][p] - sa4['starttime'][p]
                    duration2 = sa4['endtime'][p] - date_range[i]
                    duration3 = date_range[i-1] - sa4['starttime'][p]

                    if (duration1 != 0) & (duration2 != 0) & (duration3 != 0) & (duration1 > duration2) & (duration1 > duration3):
                        sa4['duration'][p] = 1 - duration2/duration1 - duration3/duration1
                        
                    else :
                        sa4['duration'][p] = 1

                sa4['amount'] = sa4['amount']*sa4['duration']

                if (sa2.empty)==False :
                    sa2.reset_index(drop=True, inplace=True)
                    sa2['duration'] = 1

                    for p in range(len(sa2)):
                        duration1 = sa2['endtime'][p] - sa2['starttime'][p]
                        duration2 = date_range[i] - sa2['starttime'][p]

                        if (duration1 != 0) & (duration2 != 0) & (duration1 > duration2):
                            sa2['duration'][p] = duration2/duration1
                        else :
                            sa2['duration'][p] = 1

                    sa2['amount'] = sa2['amount']*sa2['duration']


                    if (sa3.empty)==False :
                        sa3.reset_index(drop=True, inplace=True)
                        sa3['duration'] = 1

                        for p in range(len(sa3)):
                            duration3 = sa3['endtime'][p] - sa3['starttime'][p]
                            duration4 = sa3['endtime'][p] - date_range[i]
                            duration5 = date_range[i] - sa3['starttime'][p]

                            if (duration3 != 0) & (duration4 != 0) :
                                sa3['duration'][p] = 1 - duration4/duration3 - duration5/duration3
                            else :
                                sa3['duration'][p] = 1

                        sa3['amount'] = sa3['amount']*sa3['duration']

                
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa2_sum = sa2['amount'].sum()
                            sa3_sum = sa3['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa2_sum + sa3_sum + sa4_sum
                        else :
                            sa2_sum = sa2['amount'].sum()
                            sa3_sum = sa3['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa2_sum + sa3_sum + sa4_sum

                    else :
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa2_sum = sa2['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa2_sum + sa4_sum
                        else :
                            sa2_sum = sa2['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa2_sum + sa4_sum



                else :
                    if (sa3.empty)==False :
                        sa3.reset_index(drop=True, inplace=True)
                        sa3['duration'] = 1

                        for p in range(len(sa3)):
                            duration3 = sa3['endtime'][p] - sa3['starttime'][p]
                            duration4 = date_range[i] - sa3['starttime'][p]

                            if (duration3 != 0) & (duration4 != 0) :
                                sa3['duration'][p] = duration4/duration3
                            else :
                                sa3['duration'][p] = 1

                        sa3['amount'] = sa3['amount']*sa3['duration']

                
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa3_sum = sa3['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa3_sum + sa4_sum
                        else :
                            sa3_sum = sa3['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa3_sum + sa4_sum

                    else :
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa4_sum
                        else :
                            sa4_sum = sa4['amount'].sum()
                            
                            new_df[feature_name][i] = sa4_sum

            else:
                if (sa2.empty)==False :
                    sa2.reset_index(drop=True, inplace=True)
                    sa2['duration'] = 1

                    for p in range(len(sa2)):
                        duration1 = sa2['endtime'][p] - sa2['starttime'][p]
                        duration2 = date_range[i] - sa2['starttime'][p]

                        if (duration1 != 0) & (duration2 != 0) & (duration1 > duration2):
                            sa2['duration'][p] = duration2/duration1
                        else :
                            sa2['duration'][p] = 1

                    sa2['amount'] = sa2['amount']*sa2['duration']


                    if (sa3.empty)==False :
                        sa3.reset_index(drop=True, inplace=True)
                        sa3['duration'] = 1

                        for p in range(len(sa3)):
                            duration3 = sa3['endtime'][p] - sa3['starttime'][p]
                            duration4 = sa3['endtime'][p] - date_range[i]
                            duration5 = date_range[i] - sa3['starttime'][p]

                            if (duration3 != 0) & (duration4 != 0) :
                                sa3['duration'][p] = 1 - duration4/duration3 - duration5/duration3
                            else :
                                sa3['duration'][p] = 1

                        sa3['amount'] = sa3['amount']*sa3['duration']

                
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa2_sum = sa2['amount'].sum()
                            sa3_sum = sa3['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa2_sum + sa3_sum
                        else :
                            sa2_sum = sa2['amount'].sum()
                            sa3_sum = sa3['amount'].sum()

                            new_df[feature_name][i] = sa2_sum + sa3_sum

                    else :
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa2_sum = sa2['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa2_sum
                        else :
                            sa2_sum = sa2['amount'].sum()

                            new_df[feature_name][i] = sa2_sum 



                else :
                    if (sa3.empty)==False :
                        sa3.reset_index(drop=True, inplace=True)
                        sa3['duration'] = 1

                        for p in range(len(sa3)):
                            duration3 = sa3['endtime'][p] - sa3['starttime'][p]
                            duration4 = date_range[i] - sa3['starttime'][p]

                            if (duration3 != 0) & (duration4 != 0) :
                                sa3['duration'][p] = duration4/duration3
                            else :
                                sa3['duration'][p] = 1

                        sa3['amount'] = sa3['amount']*sa3['duration']
                
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa3_sum = sa3['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa3_sum
                        else :
                            sa3_sum = sa3['amount'].sum()

                            new_df[feature_name][i] = sa3_sum

                    else :
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()

                            new_df[feature_name][i] = sa1_sum
                        else :  
                            new_df[feature_name][i] = 0

    new_df = new_df[['subject_id', 'hadm_id', 'stay_id', 'time', feature_name]]
    new_df = new_df.reset_index(drop=True)
  
    return new_df

result_Fentanyl = Fentanyl.groupby('stay_id').apply(resampling_func2)
result_Fentanyl.reset_index(drop=True, inplace=True)

result_Fentanyl_index = result_Fentanyl[result_Fentanyl['Fentanyl']>0].index
result_Fentanyl['Fentanyl'][result_Fentanyl_index] = 1
result_Fentanyl = result_Fentanyl.reset_index(drop=True)

print('---------------------------------------------------------------------------------')
result_Fentanyl.to_csv('./10min Fentanyl.csv', index=False)

Fluids : Action

In [ ]:
'''LR_NS_id_ls = feature_ID_list['inputevents'][feature_name[21]]
LR_NS_id_ls = [int(x) for x in LR_NS_id_ls.split(',')]'''
LR_NS_id_ls = [225158,225159,225161,225823,225825,225827,225828,225941, #crystalloids
               220862,220864,220970,225168,225170,227531 #colloids and blood products
               ]


LR_NS = inputevents[inputevents['itemid'].isin(LR_NS_id_ls)]
LR_NS.reset_index(drop=True, inplace=True)

LR_NS = LR_NS[LR_NS['amount'].notna()]
condition1 = LR_NS['amount']>=0 
LR_NS = LR_NS[(condition1)]
LR_NS.reset_index(drop=True, inplace=True)
LR_NS = LR_NS.sort_values(by=['subject_id', 'hadm_id', 'stay_id', 'starttime'], ascending=[True,True,True,True])
LR_NS = LR_NS[['subject_id', 'hadm_id', 'stay_id', 'intime', 'outtime', 'starttime', 'endtime', 'amount', 'amountuom', 'rate', 'rateuom', 'ordercategorydescription']]

LR_NS['starttime'] = pd.to_datetime(LR_NS['starttime'])
LR_NS['endtime'] = pd.to_datetime(LR_NS['endtime'])
LR_NS['outtime'] = pd.to_datetime(LR_NS['outtime'])
LR_NS['intime'] = pd.to_datetime(LR_NS['intime'])

mg_index = LR_NS[LR_NS['amountuom']=='mg'].index
LR_NS['amount'][mg_index] = LR_NS['amount'][mg_index]*1000

L_index = LR_NS[LR_NS['amountuom']=='L'].index
LR_NS['amount'][L_index] = LR_NS['amount'][L_index]/1000

none_index = LR_NS[((LR_NS['rateuom']==None) | (LR_NS['rate']==None)) & (LR_NS['ordercategorydescription']!='Bolus')].index
LR_NS['rate'][none_index] = LR_NS['amount'][none_index] / ((LR_NS['endtime'][none_index] - LR_NS['starttime'][none_index]).dt.total_seconds() / 3600)

grams_min_index = LR_NS[LR_NS['rateuom']=='grams/min'].index
LR_NS['rate'][grams_min_index] = LR_NS['rate'][grams_min_index]*60

ml_min_index = LR_NS[LR_NS['rateuom']=='mL/min'].index
LR_NS['rate'][ml_min_index] = LR_NS['rate'][ml_min_index]*60

mg_min_index = LR_NS[LR_NS['rateuom']=='mg/min'].index
LR_NS['rate'][mg_min_index] = LR_NS['rate'][mg_min_index]*16.666667

mg_hour_index = LR_NS[LR_NS['rateuom']=='mg/hour'].index
LR_NS['rate'][mg_hour_index] = LR_NS['rate'][mg_hour_index]*1000

LR_NS = LR_NS[(LR_NS['ordercategorydescription']=='Bolus') | (LR_NS['rate']>=150)]
LR_NS.reset_index(drop=True, inplace=True)

LR_NS['starttime'] = pd.to_datetime(LR_NS['starttime'])
LR_NS['endtime'] = pd.to_datetime(LR_NS['endtime'])
LR_NS['outtime'] = pd.to_datetime(LR_NS['outtime'])
LR_NS['intime'] = pd.to_datetime(LR_NS['intime'])

LR_NS = LR_NS[LR_NS['starttime']<=LR_NS['endtime']]
print('LR_NS data shape: ',LR_NS.shape)
LR_NS.reset_index(drop=True, inplace=True)

#환산식
'''grams/min *60 = mL/hour
grams/hour = mL/hour
mL/min * 60 = mL/hour
mg/min * 16.666667 = mL/hour
mg/hour * 1000  = mL/hour
mcg/kg/min *  = mL/hour # exclude

mg * 1000 = mL
g = mL
L/1000 = mL
'''

In [ ]:
LR_NS['amountuom'].value_counts()

In [ ]:
LR_NS['ordercategorydescription'].value_counts()

In [ ]:
LR_NS['rateuom'].value_counts()

In [ ]:
def resampling_func2(data):
    feature_name ='LR_NS'

    column = ['subject_id','hadm_id','stay_id','time', feature_name]

    x = data.reset_index(drop=True)
    
    #x.set_index(keys='starttime', inplace=True, drop=False)

    date_range = pd.date_range(x['intime'][0], x['outtime'][0], freq='10T', inclusive='right')

    new_df = pd.DataFrame(index=date_range, columns=column)

    new_df['subject_id'] = x['subject_id'][0]
    new_df['hadm_id'] = x['hadm_id'][0]
    new_df['stay_id'] = x['stay_id'][0]
    new_df['time'] = new_df.index
    new_df[feature_name] = 0
    
    new_df = new_df.reset_index(drop=True)
    
    for i in range(len(new_df)):
        if i == 0:
            condition1 = (date_range[i]>=x['starttime'])
            condition2 = (date_range[i]>=x['endtime'])
            sa1 = x[condition1 & condition2]

            condition3 = (date_range[i]>=x['starttime'])
            condition4 = (date_range[i]<x['endtime'])
            sa2 = x[condition3 & condition4]

            if (sa2.empty)==False :
                sa2.reset_index(drop=True, inplace=True)
                sa2['duration'] = 1

                for p in range(len(sa2)):
                    duration1 = sa2['endtime'][p] - sa2['starttime'][p]
                    duration2 = date_range[i] - sa2['starttime'][p]
                    if (duration1 != 0) & (duration2 != 0) & (duration1 > duration2):
                        sa2['duration'][p] = duration2/duration1
                    else :
                        sa2['duration'][p] = 1

                sa2['amount'] = sa2['amount']*sa2['duration']
            
                if (sa1.empty)==False :
                    sa1_sum = sa1['amount'].sum()
                    sa2_sum = sa2['amount'].sum()

                    new_df[feature_name][i] = sa1_sum + sa2_sum

                else :
                    sa2_sum = sa2['amount'].sum()

                    new_df[feature_name][i] = sa2_sum

            else :
                if (sa1.empty)==False :
                    sa1_sum = sa1['amount'].sum()

                    new_df[feature_name][i] = sa1_sum

                else :
                    new_df[feature_name][i] = 0

        else :
            condition1 = ((date_range[i]>=x['starttime']) & (date_range[i-1]<x['starttime']))
            condition2 = ((date_range[i]>=x['endtime']) & (date_range[i-1]<x['endtime']))
            sa1 = x[condition1 & condition2]

            condition3 = ((date_range[i]>=x['starttime']) & (date_range[i-1]<x['starttime']))
            condition4 = ((date_range[i]<x['endtime']))
            sa2 = x[condition3 & condition4]
            
            condition5 = ((date_range[i-1]>=x['starttime']))
            condition6 = ((date_range[i]>=x['endtime']) & (date_range[i-1]<x['endtime']))
            sa3 = x[condition5 & condition6]

            condition7 = ((date_range[i-1]>=x['starttime']))
            condition8 = ((date_range[i]<x['endtime']))
            sa4 = x[condition7 & condition8]
            
            if (sa4.empty)==False :
                sa4.reset_index(drop=True, inplace=True)
                sa4['duration'] = 1

                for p in range(len(sa4)):
                    duration1 = sa4['endtime'][p] - sa4['starttime'][p]
                    duration2 = sa4['endtime'][p] - date_range[i]
                    duration3 = date_range[i-1] - sa4['starttime'][p]

                    if (duration1 != 0) & (duration2 != 0) & (duration3 != 0) & (duration1 > duration2) & (duration1 > duration3):
                        sa4['duration'][p] = 1 - duration2/duration1 - duration3/duration1
                        
                    else :
                        sa4['duration'][p] = 1

                sa4['amount'] = sa4['amount']*sa4['duration']

                if (sa2.empty)==False :
                    sa2.reset_index(drop=True, inplace=True)
                    sa2['duration'] = 1

                    for p in range(len(sa2)):
                        duration1 = sa2['endtime'][p] - sa2['starttime'][p]
                        duration2 = date_range[i] - sa2['starttime'][p]

                        if (duration1 != 0) & (duration2 != 0) & (duration1 > duration2):
                            sa2['duration'][p] = duration2/duration1
                        else :
                            sa2['duration'][p] = 1

                    sa2['amount'] = sa2['amount']*sa2['duration']


                    if (sa3.empty)==False :
                        sa3.reset_index(drop=True, inplace=True)
                        sa3['duration'] = 1

                        for p in range(len(sa3)):
                            duration3 = sa3['endtime'][p] - sa3['starttime'][p]
                            duration4 = sa3['endtime'][p] - date_range[i]
                            duration5 = date_range[i] - sa3['starttime'][p]

                            if (duration3 != 0) & (duration4 != 0) :
                                sa3['duration'][p] = 1 - duration4/duration3 - duration5/duration3
                            else :
                                sa3['duration'][p] = 1

                        sa3['amount'] = sa3['amount']*sa3['duration']

                
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa2_sum = sa2['amount'].sum()
                            sa3_sum = sa3['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa2_sum + sa3_sum + sa4_sum
                        else :
                            sa2_sum = sa2['amount'].sum()
                            sa3_sum = sa3['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa2_sum + sa3_sum + sa4_sum

                    else :
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa2_sum = sa2['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa2_sum + sa4_sum
                        else :
                            sa2_sum = sa2['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa2_sum + sa4_sum



                else :
                    if (sa3.empty)==False :
                        sa3.reset_index(drop=True, inplace=True)
                        sa3['duration'] = 1

                        for p in range(len(sa3)):
                            duration3 = sa3['endtime'][p] - sa3['starttime'][p]
                            duration4 = date_range[i] - sa3['starttime'][p]

                            if (duration3 != 0) & (duration4 != 0) :
                                sa3['duration'][p] = duration4/duration3
                            else :
                                sa3['duration'][p] = 1

                        sa3['amount'] = sa3['amount']*sa3['duration']

                
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa3_sum = sa3['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa3_sum + sa4_sum
                        else :
                            sa3_sum = sa3['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa3_sum + sa4_sum

                    else :
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa4_sum = sa4['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa4_sum
                        else :
                            sa4_sum = sa4['amount'].sum()
                            
                            new_df[feature_name][i] = sa4_sum

            else:
                if (sa2.empty)==False :
                    sa2.reset_index(drop=True, inplace=True)
                    sa2['duration'] = 1

                    for p in range(len(sa2)):
                        duration1 = sa2['endtime'][p] - sa2['starttime'][p]
                        duration2 = date_range[i] - sa2['starttime'][p]

                        if (duration1 != 0) & (duration2 != 0) & (duration1 > duration2):
                            sa2['duration'][p] = duration2/duration1
                        else :
                            sa2['duration'][p] = 1

                    sa2['amount'] = sa2['amount']*sa2['duration']


                    if (sa3.empty)==False :
                        sa3.reset_index(drop=True, inplace=True)
                        sa3['duration'] = 1

                        for p in range(len(sa3)):
                            duration3 = sa3['endtime'][p] - sa3['starttime'][p]
                            duration4 = sa3['endtime'][p] - date_range[i]
                            duration5 = date_range[i] - sa3['starttime'][p]

                            if (duration3 != 0) & (duration4 != 0) :
                                sa3['duration'][p] = 1 - duration4/duration3 - duration5/duration3
                            else :
                                sa3['duration'][p] = 1

                        sa3['amount'] = sa3['amount']*sa3['duration']

                
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa2_sum = sa2['amount'].sum()
                            sa3_sum = sa3['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa2_sum + sa3_sum
                        else :
                            sa2_sum = sa2['amount'].sum()
                            sa3_sum = sa3['amount'].sum()

                            new_df[feature_name][i] = sa2_sum + sa3_sum

                    else :
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa2_sum = sa2['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa2_sum
                        else :
                            sa2_sum = sa2['amount'].sum()

                            new_df[feature_name][i] = sa2_sum 



                else :
                    if (sa3.empty)==False :
                        sa3.reset_index(drop=True, inplace=True)
                        sa3['duration'] = 1

                        for p in range(len(sa3)):
                            duration3 = sa3['endtime'][p] - sa3['starttime'][p]
                            duration4 = date_range[i] - sa3['starttime'][p]

                            if (duration3 != 0) & (duration4 != 0) :
                                sa3['duration'][p] = duration4/duration3
                            else :
                                sa3['duration'][p] = 1

                        sa3['amount'] = sa3['amount']*sa3['duration']
                
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()
                            sa3_sum = sa3['amount'].sum()

                            new_df[feature_name][i] = sa1_sum + sa3_sum
                        else :
                            sa3_sum = sa3['amount'].sum()

                            new_df[feature_name][i] = sa3_sum

                    else :
                        if (sa1.empty)==False :
                            sa1_sum = sa1['amount'].sum()

                            new_df[feature_name][i] = sa1_sum
                        else :  
                            new_df[feature_name][i] = 0

    new_df = new_df[['subject_id', 'hadm_id', 'stay_id', 'time', feature_name]]
    new_df = new_df.reset_index(drop=True)
  
    return new_df

result_LR_NS = LR_NS.groupby('stay_id').apply(resampling_func2)
result_LR_NS.reset_index(drop=True, inplace=True)

print('---------------------------------------------------------------------------------')
result_LR_NS.to_csv('./10min LR_NS_TR_AL.csv', index=False)